In [230]:
import numpy as np
import random


categorical_values = np.array([1, 2, 3, 4, 5])
p = [0.4, 0.4, 0.1, 0.06, 0.04] # distribution of random generated categorical values
# p = None 
target_average = 1.32 
atol = 0.02 # target average toleration
target_std = 0.4 #target_standard deviation
population_size = 26


def close_to_mean(sample, target_average, atol, p ):
    cnt = 0
    while not np.isclose(np.mean(sample), target_average, atol):
        if np.mean(sample) > target_average:
            arr_to_replace_lenght = len(sample[sample > target_average])
            arr_to_replace = np.random.choice(categorical_values, size= arr_to_replace_lenght, p=p)
            np.place(sample,sample > target_average, arr_to_replace)
        else:
            arr_to_replace_lenght = len(sample[sample < target_average])
            arr_to_replace = np.random.choice(categorical_values, arr_to_replace_lenght, p=p)
            np.place(sample,sample < target_average, arr_to_replace)

        cnt += 1
        if cnt > 10_000:
            print("too much iterations atol")
            break
        elif cnt % 200 == 0 :
            atol *= 1.01             
    return sample, atol 

# Initialize and guess the sample 
sample = np.random.choice(categorical_values, population_size, p=p) # type: ignore
sample, _ = close_to_mean(sample, target_average, atol, p)

cnt = 0
while np.std(sample) > target_std:
    outliers = sample[(sample > np.mean(sample) + np.std(sample)) | (sample < np.mean(sample) - np.std(sample))]
    mask = np.isin(sample, outliers, invert=True)
    not_outliers  = sample[mask]    
    arr_to_replace_lenght = len(outliers)
    arr_to_replace = np.random.choice(categorical_values, arr_to_replace_lenght)
    sample = np.concatenate((not_outliers, arr_to_replace))
    sample, _ = close_to_mean(sample, target_average, atol, p)

    cnt += 1
    if cnt > 10_000:
        print("too much iterations target_std")
        break
    elif cnt % 200 == 0 :
        target_std *= 1.01


    
     

sample = np.sort(sample)
print("Sample:", sample)
print("Average:", np.mean(sample))
print("Std dev:", np.std(sample))

Sample: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
Average: 1.3076923076923077
Std dev: 0.46153846153846156


In [233]:
# Set the target mean and desired standard deviation
target_mean = 50  # for example
std_dev = 10      # standard deviation

# Set the size of the sample
sample_size = 26

# Generate the random sample
sample = np.random.normal(target_mean, std_dev, sample_size)

# Check the mean of the generated sample
actual_mean = np.mean(sample)

print("Generated sample:", sample)
print("Generated sample mean:", actual_mean)

Generated sample: [46.48883787 51.66295445 46.45762597 35.35081292 28.85517363 51.99884295
 52.45154655 45.20999709 51.41706626 45.90738226 42.36478982 51.28005934
 53.22680366 53.78816861 48.58992694 77.55289871 31.22624325 60.37044003
 41.9889611  46.79277928 50.21844535 22.09877564 42.29992368 58.66703512
 38.15781683 39.43017993]
Generated sample mean: 46.686672586592934


In [240]:
import numpy as np
from scipy.optimize import minimize

# Define the target mean
target_mean = 2.2

# Define the categories
categories = np.array([1, 2, 3, 4, 5])

# Function to calculate the difference from the target mean
def mean_difference(probabilities):
    expected_mean = np.sum(probabilities * categories)
    return (expected_mean - target_mean) ** 2

# Constraints: sum of probabilities must be 1
cons = ({'type': 'eq', 'fun': lambda p: np.sum(p) - 1})

# Bounds for each probability: must be between 0 and 1
bounds = [(0, 1) for _ in categories]

# Initial guess (uniform distribution)
initial_probabilities = np.ones(len(categories)) / len(categories)

# Use minimize function to find the best probabilities
result = minimize(mean_difference, initial_probabilities, bounds=bounds, constraints=cons)

# Check if the optimizer found a solution
if result.success:
    optimized_probabilities = result.x
    print("Optimized Probabilities:", optimized_probabilities)
    print("Expected Mean:", np.sum(optimized_probabilities * categories))
else:
    print("Optimization failed:", result.message)

Optimized Probabilities: [0.52 0.12 0.12 0.12 0.12]
Expected Mean: 2.199999992474921


In [243]:
probabilities = np.array([0.15, 0.25, 0.3, 0.2, 0.15])

# Ensure probabilities sum to 1
probabilities /= probabilities.sum()

probabilities

array([0.14285714, 0.23809524, 0.28571429, 0.19047619, 0.14285714])

In [250]:
probabilities = np.array([5, 5, 1, 1, 1])

print(probabilities.sum())

# Ensure probabilities sum to 1
probabilities = probabilities / probabilities.sum()

print(probabilities)
print(probabilities.sum())

13
[0.38461538 0.38461538 0.07692308 0.07692308 0.07692308]
1.0
